In [2]:
import urllib
from urllib.request import urlopen
import pandas as pd

file = urllib.request.urlopen('https://alexip-ml.s3.amazonaws.com/stackexchange_812k.csv.gz')
df = pd.read_csv(file, compression='gzip', header=0, sep=',', quotechar='"')

In [3]:
df.tail(10)

,post_id,parent_id,comment_id,text,category
812122,279993,NaN,536604.0,@Calimo i just plotted the results of each con...,comment
812123,279993,NaN,536608.0,"Ok, but then that's not a ROC curve, which sho...",comment
812124,279993,NaN,536612.0,@Calimo yes i understand. but therein lies my ...,comment
812125,279993,NaN,536451.0,You really want a cost function...,comment
812126,279993,NaN,536454.0,"@Calimo it would seem so, stumbled upon `What ...",comment
812127,279994,NaN,536471.0,"It does run, and gives very valid looking esti...",comment
812128,279998,NaN,536439.0,It seems to me that you are correct; the doubl...,comment
812129,279998,NaN,536514.0,It wouldn't be the first time a grader has mis...,comment
812130,279999,NaN,536802.0,The basic idea is to compare the clustering co...,comment
812131,279999,NaN,542550.0,"As per your other question, your data does not...",comment


In [4]:
import re

# Remove HTML tags
df.text = df.text.str.replace('<[^<]+?>', '')

# Remove Latex
df.text = df.text.str.replace('\\textbf{([^}]*)}', '')

# Remove rows with empty text
filter = df["text"] != ""
df = df[filter]

# Remove 100 words or longer
df = df[df['text'].str.split().str.len().lt(100)]

# Remove 5 words or less
df = df[df['text'].str.split().str.len().gt(5)]

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

# from: https://stackoverflow.com/a/54398984/773263
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


df['cleanText']=df['text'].map(lambda s:preprocess(s)) 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df.head(15)